In [24]:
from sklearn.linear_model import ARDRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from Importer import *
import Processor
importlib.reload(Processor)

<module 'Processor' from 'C:\\Users\\venablecrofta\\OneDrive - Virginia Tech\\Documents\\GitHub\\ME_5984-Final-Project\\Processor.py'>

In [51]:
path =  r"C:\Users\venablecrofta\OneDrive - Virginia Tech\Documents\GitHub\ME_5984-Final-Project\train_FD001.csv"
test_path =  r"C:\Users\venablecrofta\OneDrive - Virginia Tech\Documents\GitHub\ME_5984-Final-Project\test_FD001.csv"

index_names  = ['id', 'cycles']
setting_names  = ['setting_1', 'setting_2', 'setting_3']
sensor_names = [f"s_{x}" for x in range(1, 22)]
col_names = index_names + setting_names + sensor_names

def apply_names (path, column_names):
    df  = pd.read_csv(path, header=None, delimiter =r"\s+", names = column_names)
#     df = df.set_axis(column_names, axis=1)
    return df
df_train = apply_names(path, col_names)
df_test= apply_names(test_path, col_names)


df_train_RUL = df_train.groupby(['id']).agg({'cycles':'max'})
df_train_RUL.rename(columns={'cycles':'life'},inplace=True)
df_train_RUL.head()
df_train=df_train.merge(df_train_RUL,how='left',on=['id'])

df_train['RUL']=df_train['life']-df_train['cycles']
df_train.drop(['life'],axis=1,inplace=True)
df_train.head()

,id,cycles,setting_1,setting_2,setting_3,s_1,s_2,s_3,s_4,s_5,...,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


In [52]:
df_test_RUL = pd.read_csv('RUL_FD001.csv',header=None, delimiter =r"\s+")
print(df_test_RUL)

      0
0   112
1    98
2    69
3    82
4    91
..  ...
95  137
96   82
97   59
98  117
99   20

[100 rows x 1 columns]


In [53]:
columns_to_drop = ['setting_1', 'setting_2', 'setting_3', 's_1', 's_5', 's_6', 's_10', 's_16', 's_18', 's_19']
df_train.drop(columns=columns_to_drop, inplace=True, errors='ignore')
df_test.drop(columns=columns_to_drop, inplace=True, errors='ignore')
df_test.head()

,id,cycles,s_2,s_3,s_4,s_7,s_8,s_9,s_11,s_12,s_13,s_14,s_15,s_17,s_20,s_21
0,1,1,643.02,1585.29,1398.21,553.90,2388.04,9050.17,47.20,521.72,2388.03,8125.55,8.4052,392,38.86,23.3735
1,1,2,641.71,1588.45,1395.42,554.85,2388.01,9054.42,47.50,522.16,2388.06,8139.62,8.3803,393,39.02,23.3916
2,1,3,642.46,1586.94,1401.34,554.11,2388.05,9056.96,47.50,521.97,2388.03,8130.10,8.4441,393,39.08,23.4166
3,1,4,642.44,1584.12,1406.42,554.07,2388.03,9045.29,47.28,521.38,2388.05,8132.90,8.3917,391,39.00,23.3737
4,1,5,642.51,1587.19,1401.92,554.16,2388.01,9044.55,47.31,522.15,2388.03,8129.54,8.4031,390,38.99,23.4130


In [58]:
scaler = MinMaxScaler()
df_test_RUL = df_test.groupby(['id']).agg({'cycles':'max'})
df_test_RUL.rename(columns={'cycles':'life'},inplace=True)
df_test=df_test.merge(df_test_RUL,how='left',on=['id'])
df_test['RUL']=df_test['life']-df_test['cycles']
df_test.drop(['life'],axis=1,inplace=True)
print(df_test)

        id  cycles     s_2      s_3      s_4     s_7      s_8      s_9   s_11  \
0        1       1  643.02  1585.29  1398.21  553.90  2388.04  9050.17  47.20   
1        1       2  641.71  1588.45  1395.42  554.85  2388.01  9054.42  47.50   
2        1       3  642.46  1586.94  1401.34  554.11  2388.05  9056.96  47.50   
3        1       4  642.44  1584.12  1406.42  554.07  2388.03  9045.29  47.28   
4        1       5  642.51  1587.19  1401.92  554.16  2388.01  9044.55  47.31   
...    ...     ...     ...      ...      ...     ...      ...      ...    ...   
13091  100     194  643.24  1599.45  1415.79  553.41  2388.02  9142.37  47.69   
13092  100     195  643.22  1595.69  1422.05  553.22  2388.05  9140.68  47.60   
13093  100     196  643.44  1593.15  1406.82  553.04  2388.11  9146.81  47.57   
13094  100     197  643.26  1594.99  1419.36  553.37  2388.07  9148.85  47.61   
13095  100     198  642.95  1601.62  1424.99  552.48  2388.06  9155.03  47.80   

         s_12     s_13     

In [60]:
df_engine1 = df_train[df_train["id"] == 1]  # Adjust column name if needed
# 2. Select a sensor column (e.g., 'sensor_1')
sensor_col = "s_2"  # replace with your actual column name

# 3. Get the full time-series for that sensor
X_train = df_engine1[sensor_col].values.reshape(1, -1)  # shape: (1, timesteps)

# 4. Assign a target value (e.g., true RUL at t=0)
y_train = np.array([df_train[df_train["id"] == 1]["RUL"].values])  # shape: (1,)

df_engine2 = df_test[df_test["id"] == 1]

x_test = df_engine2[sensor_col].values.reshape(1,-1)

y_test = np.array([df_test[df_test["id"] == 1]["RUL"].values])  # shape: (1,)


In [ ]:
X = np.vstack([X_train**i for i in range(2)]).T
print(np.shape(X))
alpha = 100 # Precision of prior
beta = 1 / (0.05 ** 2)  

S_N_inv = alpha * np.eye(X.shape[1]) + beta * X.T @ X  # Correct variable name
m_N = beta * np.linalg.solve(S_N_inv, X.T @ y_train)
# Compute posterior distribution
# S_N_inv = alpha * np.eye(5) + beta * X.T @ X
S_N = np.linalg.inv(S_N_inv)
# m_N = beta * S_N @ X.T @ y_noisy


x_test = np.linspace(-1, 1, 100)
X_test = np.vstack([x_test**i for i in range(3)]).T

y_pred_mean = X_test @ m_N
y_pred_var = 1 / beta + np.sum(X_test @ S_N * X_test, axis=1)
y_pred_std = np.sqrt(y_pred_var)

# Compute log-likelihood
sigma = 1 / beta
residuals = y_train - X @ m_N
log_likelihood = -0.5 * len(y_train) * np.log(2 * np.pi * sigma) - (0.5 / sigma) * np.sum(residuals**2)


# Compute BIC
num_params = X.shape[1]
num_data = len(y_train)
bic_score = -2 * log_likelihood + num_params * np.log(num_data)
print(f"BIC Score: {bic_score}")

# Plot results
plt.figure(figsize=(8,6))
plt.scatter(x, y_noisy, color='red', label='Noisy Data')
plt.plot(x_test, y_pred_mean, label='Mean Prediction', color='blue')
plt.plot(x, y_true, color = 'green', label = 'True Equation')
plt.fill_between(x_test, y_pred_mean - y_pred_std, y_pred_mean + y_pred_std, 
                 color='blue', alpha=0.2, label='Predictive Std. Dev.')
plt.legend()
plt.title("Bayesian Polynomial Regression (Degree-4)")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [67]:
n_engines = 50
cycles_per_engine = np.random.randint(10, 100, size=n_engines)
sensor_data = [np.random.randn(n, 3) for n in cycles_per_engine]  # 3 sensors
y = [np.random.randn(n) for n in cycles_per_engine]  # Degradation metric

# Flatten data and create engine indices
engine_idx = np.repeat(np.arange(n_engines), cycles_per_engine)
X = np.vstack(sensor_data)

with pm.Model() as hierarchical_model:
    # Hyperpriors for random effects
    tau = pm.HalfNormal("tau", sigma=5, shape=3)  # One per sensor
    
    # Random intercepts and slopes per engine
    u0 = pm.Normal("u0", mu=0, sigma=tau[0], shape=n_engines)
    u1 = pm.Normal("u1", mu=0, sigma=tau[1], shape=n_engines)
    
    # Fixed effects (population-level)
    beta = pm.Normal("beta", mu=0, sigma=10, shape=3)
    
    # Likelihood
    mu = beta[0] + beta[1] * X[:, 0] + beta[2] * X[:, 1] + u0[engine_idx] + u1[engine_idx] * X[:, 2]
    sigma = pm.HalfNormal("sigma", sigma=1)
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=np.concatenate(y))
    
    trace = pm.sample(2000, tune=1000)

(14, 41262)


MemoryError: Unable to allocate 12.7 GiB for an array with shape (41262, 41262) and data type float64

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 100)            46000     
                                                                 
 batch_normalization (Batch  (None, 1, 100)            400       
 Normalization)                                                  
                                                                 
 lstm_1 (LSTM)               (None, 1, 50)             30200     
                                                                 
 dropout (Dropout)           (None, 1, 50)             0         
                                                                 
 lstm_2 (LSTM)               (None, 1, 10)             2440      
                                                                 
 dropout_1 (Dropout)         (None, 1, 10)             0         
                                                        

R-squared error: -295.03%
Root Mean Squared Error: 105.98
